In [1]:
%load_ext tensorboard

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, concatenate, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
import pandas as pd
import numpy as np
from PIL import Image
from math import floor
import matplotlib.pyplot as plt
import datetime

# Clear logs from previous runs
# %rm -rf ../logs/

/home/fri/UAV_Team/ego-planner/src/neural_net/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/fri/UAV_Team/ego-planner/src/neural_net/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/fri/UAV_Team/ego-planner/src/neural_net/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

In [2]:
NUM_INPUTS = 13 # used to be 13
NUM_OUTPUTS = 9

img_shape = (480, 640)

# Load the dataset input and output
inputNums = np.loadtxt('../dataset/0_input.csv', delimiter=',')#, usecols=tuple(range(NUM_INPUTS))) # (num training points, 13)
y = np.loadtxt('../dataset/0_output.csv', delimiter=',', usecols=tuple(range(NUM_OUTPUTS))) # (num training points, 11)
X = np.zeros((len(inputNums), *img_shape, 1)) # (num training points, 480, 640)

# For each of the training examples there should be exactly 1 image
for i in range(0, len(inputNums)):
    image = Image.open('../dataset/img_' + str(i) + '.jpg') 
    imageArr = np.asarray(image).reshape(*img_shape, 1) # Converts the image into a 2D 480*640 array

    X[i] = imageArr / 255

In [3]:
max_mag = 2.0

print(inputNums.shape);

def mag(x, y):
    return np.sqrt(x * x + y * y)

for inputRow in inputNums:
    m = mag(*inputRow[0:2]) # constrain goal positions
    if m > max_mag:
        inputRow[0] = inputRow[0] * max_mag / m 
        inputRow[1] = inputRow[1] * max_mag / m 

(9583, 13)


Let's view some of our data images

In [5]:
num_show = 10

# fig = plt.figure(figsize=(480/20, 640/20), dpi=100)
# for i in range(num_show):
#     ax = fig.add_subplot(num_show, 1, i+1)
#     plt.imshow(X[i])

Next comes actually training on the data. 
PS. If you want to view the tensorboard data, run `tensorboard --logdir logs/fit` in another window.

In [ ]:
numerical_inputs = Input(shape=(NUM_INPUTS,))
image_inputs = Input(shape=(*img_shape, 1))

# Process the image data first
x_image = Conv2D(32, (7, 7), strides=(4, 4), padding='same', activation='relu')(image_inputs)
x_image = Conv2D(64, (5, 5), strides=(4, 4), padding="same", activation="relu")(x_image)
x_image = Conv2D(128, (3, 3), strides=(2, 2), padding="same", activation="relu")(x_image)
x_image = Conv2D(128, (3, 3), strides=(2, 2), padding="same", activation="relu")(x_image)
x_image = Conv2D(256, (3, 3), strides=(2, 2), padding="same", activation="relu")(x_image)
x_image = Model(image_inputs, x_image)
x_image = Flatten()(x_image.output)

print(x_image.shape)
print(numerical_inputs.shape)

# Combine the latent space representation with the numerical input
combined = concatenate([x_image, numerical_inputs])
print(combined.shape)
x_combined = Dense(300, kernel_regularizer=l2(0.01), activation='relu')(combined)
x_combined = Dense(60, kernel_regularizer=l2(0.01), activation='relu')(x_combined)
outputs = Dense(NUM_OUTPUTS, kernel_regularizer=l2(0.01), activation='linear')(x_combined)

model = Model([image_inputs, numerical_inputs], outputs)

model.compile(loss='mse', optimizer='adam', metrics=['mae'])

log_dir = "../logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit([X, inputNums], y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[tensorboard_callback])

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

(?, 5120)
(?, 13)
(?, 5133)
Train on 7666 samples, validate on 1917 samples
Epoch 1/100
7666/7666 [==============================] - 66s 9ms/sample - loss: 21.2094 - mean_absolute_error: 2.1887 - val_loss: 44.9000 - val_mean_absolute_error: 2.8595
Epoch 2/100
7666/7666 [==============================] - 65s 8ms/sample - loss: 11.4187 - mean_absolute_error: 1.4576 - val_loss: 42.4568 - val_mean_absolute_error: 2.6882
Epoch 3/100
7666/7666 [==============================] - 65s 8ms/sample - loss: 5.7221 - mean_absolute_error: 0.9319 - val_loss: 47.0158 - val_mean_absolute_error: 2.8165
Epoch 4/100
6688/7666 [=========================>....] - ETA: 5s - loss: 3.5465 - mean_absolute_error: 0.7040

In [ ]:
# Save the model
# model.save('models/conv_model')
model_dir = '../models/conv_model_output_culled'
tf.saved_model.save(model, model_dir)

# Open Tensorboard (actually, run this on the command line on your Terminal)
# %tensorboard --logdir ../logs/fit

In [ ]:
# Load the model
loaded_model = model_load('../models')
loaded_model.fit([X, inputNums], y, epochs=100, batch_size=32, validation_split=0.2)